<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/polyhedron-gdl/halloween-seminar-2023/blob/main/1_notebooks/chapter-11-01.ipynb">
        <img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
</table>

# Summarize PDF Document with OpenAI

This notebook is based on the blog [The Ultimate Guide to PDF Summarization with OpenAI API: Simplify Your Reading Process](https://medium.com/@kapildevkhatik2/the-ultimate-guide-to-pdf-summarization-with-openai-api-simplify-your-reading-process-80021210cd11#id_token=eyJhbGciOiJSUzI1NiIsImtpZCI6IjA1MTUwYTEzMjBiOTM5NWIwNTcxNjg3NzM3NjkyODUwOWJhYjQ0YWMiLCJ0eXAiOiJKV1QifQ.eyJpc3MiOiJodHRwczovL2FjY291bnRzLmdvb2dsZS5jb20iLCJuYmYiOjE2ODcxNzUzNzMsImF1ZCI6IjIxNjI5NjAzNTgzNC1rMWs2cWUwNjBzMnRwMmEyamFtNGxqZGNtczAwc3R0Zy5hcHBzLmdvb2dsZXVzZXJjb250ZW50LmNvbSIsInN1YiI6IjEwMTY0ODk4MDkzODU4MzAwMTMyMSIsImVtYWlsIjoiZ2lvdmFubmkuZGVsbGFsdW5nYUBnbWFpbC5jb20iLCJlbWFpbF92ZXJpZmllZCI6dHJ1ZSwiYXpwIjoiMjE2Mjk2MDM1ODM0LWsxazZxZTA2MHMydHAyYTJqYW00bGpkY21zMDBzdHRnLmFwcHMuZ29vZ2xldXNlcmNvbnRlbnQuY29tIiwibmFtZSI6Ikdpb3Zhbm5pIERlbGxhIEx1bmdhIiwicGljdHVyZSI6Imh0dHBzOi8vbGgzLmdvb2dsZXVzZXJjb250ZW50LmNvbS9hL0FBY0hUdGVzWVZMY1dmZkphY2ZfSmU4WWpvbHF4QUNTSTN3MHdsWlJRWWxjdXc9czk2LWMiLCJnaXZlbl9uYW1lIjoiR2lvdmFubmkiLCJmYW1pbHlfbmFtZSI6IkRlbGxhIEx1bmdhIiwiaWF0IjoxNjg3MTc1NjczLCJleHAiOjE2ODcxNzkyNzMsImp0aSI6ImM3NTBkZjNiNTBiODA4NmZhM2YwMTI3OTY4MDFmMzUyMjIyOGIzYTEifQ.W6vLMMjlVXO5upLzoW64ctuSV-fukd1n7WSawt5qGCE_5ThBHpi5FgFiFnRXKy5zAmnAawI5CDQ8YgQe14WuZcSa2jpQPfcmhI64LZIXR-KTmxta4OWKYWCavmYwaoG9SyrLEPr7OA2nw2ye9lQB0sfZqVWmYehVs04GR5etWEKEtgPkm1DLYb2aTPNCvrAyXjUqUhF29Uy8vQZ3OsFhGMB2cfMJHqGpQJMcCmJ3TCEDJlNOImzSWyVTLZx9pgbcYa5QtMCeOuWbF25MpQBSs-SGUB8PAqTg6XF_QbTe0iwzzsA0hfPsLMCxpk28Ji5GRvkV1V31xJDj5DpSx0Kukw) by Kapil Khatik. Let's start with the basic import...

In [3]:
import os
import fitz
import openai
import json

from io import StringIO

In [4]:
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

- **Fitz**: Fitz is a Python library for working with PDF files. It allows you to read, write, and manipulate PDF documents in your Python scripts.

- **OpenAI**: OpenAI is a powerful AI platform that provides access to state-of-the-art language models, such as GPT-3, for natural language processing tasks.

In [5]:
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize

In [7]:
path = './data/'
filename = 'Guidelines on IRRBB and CSRBB.pdf'

# Join the path and filename
filename = os.path.join(path, filename)

# Print the resulting filepath
print(filename)

./data/Guidelines on IRRBB and CSRBB.pdf


## Step 1: Reading the Text Content of a PDF File with Fitz in Python

See the slides for a full description of the code

In [8]:
context = ""

# Open the PDF file
with fitz.open(filename) as pdf_file:
    # Get the number of pages in the PDF file
    num_pages = pdf_file.page_count

    # Loop through each page in the PDF file
    for page_num in range(num_pages):
        # Get the current page
        page = pdf_file[page_num]

        # Get the text from the current page
        page_text = page.get_text()
  
        # Append the text to context
        context += page_text

## Step 2: Breaking Down Lengthy PDF Text into Manageable Chunks for Summarization

The next step involves feeding the context into the “split_text” function, which is responsible for dividing the text into paragraphs containing 5000 characters each.

In [9]:
def split_text(text, chunk_size=5000):
    
    '''
    Splits the given text into chunks of approximately the specified chunk size.

    Args:
    text (str): The text to split.

    chunk_size (int): The desired size of each chunk (in characters).

    Returns:
    List[str]: A list of chunks, each of approximately the specified chunk size.
    '''

    chunks = []
    current_chunk = StringIO()
    current_size = 0
    sentences = sent_tokenize(text)
    for sentence in sentences:
        sentence_size = len(sentence)
        if sentence_size > chunk_size:
            while sentence_size > chunk_size:
                chunk = sentence[:chunk_size]
                chunks.append(chunk)
                sentence = sentence[chunk_size:]
                sentence_size -= chunk_size
                current_chunk = StringIO()
                current_size = 0
        if current_size + sentence_size < chunk_size:
                current_chunk.write(sentence)
                current_size += sentence_size
        else:
                chunks.append(current_chunk.getvalue())
                current_chunk = StringIO()
                current_chunk.write(sentence)
                current_size = sentence_size
    if current_chunk:
        chunks.append(current_chunk.getvalue())

    return chunks

The code defines a Python function named `split_text` that takes a lengthy text and splits it into smaller chunks of approximately the same size. The function uses the `sent_tokenize` function to split the text into sentences and then iterates over each sentence to create the chunks. The size of each chunk can be specified in the function arguments, with the default size being 5000 characters. The function returns a list of chunks, with each chunk having approximately the same size as specified. This function can be used for summarizing lengthy PDF texts by breaking them down into smaller chunks that can be easily processed.

In summary, the `split_text` function provides a convenient way to break down lengthy PDF texts into manageable chunks for summarization, making it easier to process and extract key information.

In [10]:
def remove_newlines(serie):
    serie = serie.replace('\n', ' ')
    serie = serie.replace('\\n', ' ')
    serie = serie.replace('  ', ' ')
    serie = serie.replace('  ', ' ')
    return serie

In [11]:
context = remove_newlines(context)

In [12]:
split_text(context, 1000)[3]

'These new Guidelines maintain continuity with the previous Guidelines as far as possible, while updating some elements.The Guidelines are broadly consistent with the Basel standards with some further elaborated sections following the CRD mandate, particularly on CSRBB assessment and monitoring and non- satisfactory IRRBB internal systems.The EBA is mandated to specify in these Guidelines additional criteria for the assessment and monitoring by institutions of their credit spread risk arising from their non-trading book activities (CSRBB).The Guidelines provide a definition and the scope of application of CSRBB.They contain dedicated sections for CSRBB with specific provisions on the identification, assessment and monitoring of CSRBB.'

## 3. Generating summaries of the text chunks using OpenAI’s language models.

The following code defines a function named `gpt3_completion` that takes in a prompt string and four optional parameters: `engine`, `temp`, `top_p`, and `tokens`.

- `prompt = prompt.encode(encoding='ASCII',errors='ignore').decode()` encodes the prompt string to ASCII format and ignores any errors that may arise during encoding, and then decodes it back to a string.

- `try:` and `except Exception as oops:` are used to handle any errors that may occur while executing the code inside the try block.

- `response = openai.Completion.create(...)` sends a request to the OpenAI GPT-3 API to generate text completion based on the provided prompt, engine, temp, top_p, and tokens parameters. The response is stored in the response variable.

- `return response.choices[0].text.strip()` extracts the generated text from the response object and removes any leading or trailing white spaces.

If an error occurs during the execution of the try block, the function returns an error message containing the error information.

In [13]:
def gpt3_completion(  prompt
                    , engine='text-davinci-003'
                    , temp=0.5
                    , top_p=0.3
                    , tokens=1000):

    prompt = prompt.encode(encoding='ASCII',errors='ignore').decode()
    try:
        response = openai.Completion.create(
        engine=engine,
        prompt=prompt,
        temperature=temp,
        top_p=top_p,
        max_tokens=tokens
        )
        return response.choices[0].text.strip()
    except Exception as oops:
        return "GPT-3 error: %s" % oops

In [14]:
def summarize(document):
  
    # Calling the split function to split text
    chunks = split_text(document)
  
    summaries = []
    for chunk in chunks:
        prompt = "Please summarize the following document: \n"
        summary = gpt3_completion(prompt + chunk)
        if summary.startswith("GPT-3 error:"):
            continue
        summaries.append(summary)
  
    return "".join(summaries)

In [15]:
document = context

# Call the summrize function with the document as input
summary = summarize(document)

In [16]:
print(len(document))

213034


In [17]:
print(len(summary))

46628


In [17]:
path       = './data/'
filename_1 = 'Guidelines on IRRBB and CSRBB.txt'
filename_2 = 'Guidelines on IRRBB and CSRBB - Summary.txt'

# Join the path and filename
filename_1 = os.path.join(path, filename_1)
filename_2 = os.path.join(path, filename_2)

# Print the resulting filepath
print('\n', filename_1, '\n', filename_2)


 ./data/Guidelines on IRRBB and CSRBB.txt 
 ./data/Guidelines on IRRBB and CSRBB - Summary.txt


In [18]:
# Define some text content to be saved
text_content = document

# Open a new file for writing using the UTF-8 encoding
with open(filename_1, "w", encoding="utf-8") as f:
    # Write the text content to the file
    f.write(text_content)

In [19]:
# Define some text content to be saved
text_content = summary

# Open a new file for writing using the UTF-8 encoding
with open(filename_2, "w", encoding="utf-8") as f:
    # Write the text content to the file
    f.write(text_content)